### Goals
- find common stats between states
- combine useful information between schools for analysis

In [1]:
import os

import pandas as pd

In [2]:
# denormalize nys school-level data
nys_grad_rate = pd.read_csv('../prepared_data/nys_grad_rate.csv')

In [4]:
# 4 year results for all students for every school
nys_grad_rate_by_school = nys_grad_rate[
    (nys_grad_rate['AGGREGATION_TYPE'] == 'School') & 
    (nys_grad_rate['SUBGROUP_NAME'] == 'All Students') &
    (nys_grad_rate['MEMBERSHIP_DESC'] == '2013 Total Cohort - 4 Year Outcome') 
]
# note: a few schools are still in here twice. also, there is at least
# one case of 2 schools with the same name (BEACON HIGH SCHOOL)
nys_grad_rate_by_school.drop_duplicates(inplace=True)

/Users/josh.laurito/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [5]:
# nys report card data tables
nys_reportcard_tables = [x for x in os.listdir('../prepared_data/') if x.find('nys_report') > -1]


In [240]:
# function to define whether a row is school-level data
def is_school(entity_cd):
    id_string = str(entity_cd)
    if len(id_string) < 12:
        return False
    elif id_string[-4:] == '0000':
        return False
    elif id_string == '111111111111':
        return False
    else:
        return True

# function to generate a shortname from tablename:
def shortname(s):
    _ = s.lower()
    strings_to_replace= ['(',')','&','-','nys_reportcard_','.csv','results']
    for st in strings_to_replace:
        _ = _.replace(st, '')
    _ = '_'.join(_.split('_')[:4])
    if _[-1] == '_':
        _ = _[:-1]
    return _

# function to see if item is in list
def list_contains(my_list, item):
    try:
        my_list.index(item)
        return True
    except ValueError:
        return False

In [254]:
# denormalize NYS reportcard data
tables_to_skip = [
    'new_york_state_alternate',
    'accountability',
    'regents_examination_annual',
    'nyseslat_annual',
    'regents_competency_test_rct',
    'regents_common_core_examination',
    'recently_arrived_ell_students'
]

if 'nys_denormalized' in vars():
    del nys_denormalized

for t in nys_reportcard_tables:
    table_shortname = shortname(t)
    _df = pd.read_csv('../prepared_data/' + t)
    
    # logic to account for naming inconsistencies
    # and only gathering school-level data and
    # all-student data when appropriate
    if table_shortname == 'accountability':
        _df.rename( columns={'ACC_YEAR': 'YEAR', 'ACC_NAME': 'SUBGROUP_NAME'}, inplace=True)
    if table_shortname == 'institution_grouping' or table_shortname[:13] == 'total_cohort_':
        _df['YEAR'] = 2016
    if table_shortname[:13] == 'total_cohort_':
        single_cohort = _df[_df['COHORT'] == 2013]
        if len(single_cohort) > 0:
            _df = single_cohort
        else:
            _df = _df[_df['COHORT'] == 2012]
    if table_shortname == 'boces_and_nrc' or table_shortname == 'staff':
        _df.rename( columns={'SCHOOL_NAME': 'ENTITY_NAME'}, inplace=True)
    if _df.columns.contains('SUBGROUP_NAME'):
        _df = _df[_df['SUBGROUP_NAME'] == 'All Students']
        _df.drop('SUBGROUP_NAME', axis=1, inplace = True)
        
    # data cleaning for all tables
    _df = _df[_df['ENTITY_CD'].apply(lambda x: is_school(x))]
    _df = _df[_df['YEAR'] == 2016]
    _df.drop_duplicates(inplace=True)
    
    # initialization and table skipping
    if 'nys_denormalized' not in vars():
        nys_denormalized = _df
        print('initializing table with ' + table_shortname)
    elif list_contains(tables_to_skip, table_shortname):
        print('skipping table ' + table_shortname)
    else:
        print('trying to join ' + table_shortname )
        _tmp = nys_denormalized.merge(
            right = _df, 
            how = 'outer',
            on = ['ENTITY_CD','ENTITY_NAME', 'YEAR'],
            suffixes = ['', '_' + table_shortname]
        )
        nys_denormalized = _tmp
    del _df
print('all tables joined!')

initializing table with science8_subgroup
trying to join total_cohort_global_hist
trying to join attendance_and_suspensions


/Users/josh.laurito/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


skipping table new_york_state_alternate
trying to join institution_grouping
skipping table regents_examination_annual
trying to join boces_and_nrc


/Users/josh.laurito/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6,7,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


skipping table accountability
trying to join ela5_subgroup
trying to join total_cohort_us_hist
trying to join math8_subgroup
trying to join math7_subgroup
trying to join average_class_size
skipping table nyseslat_annual
trying to join ela4_subgroup
trying to join beds_day_enrollment_by
trying to join math3_subgroup
trying to join math6_subgroup
trying to join math5_subgroup
skipping table regents_competency_test_rct
trying to join staff
trying to join ela8_subgroup
trying to join ela7_subgroup
trying to join total_cohort_graduation_rate
trying to join high_school_postgraduation_plans
trying to join beds_day_enrollment
trying to join math4_subgroup
trying to join demographic_factors
trying to join high_school_completers
skipping table regents_common_core_examination
skipping table recently_arrived_ell_students
trying to join ela6_subgroup
trying to join science4_subgroup
trying to join ela3_subgroup
trying to join total_cohort_ela_subgroup
trying to join total_cohort_science_subgroup
tr

In [255]:
#_df = pd.read_csv('../prepared_data/nys_reportcard_Total_Cohort_Graduation_Rate_for_Advanced_Students.csv')
#nys_denormalized[nys_denormalized['ENTITY_NAME'] =='ESPERANZA PREPATORY ACADEMY'].to_csv('tmp.csv')
#nys_denormalized.head().iloc[2][60:100]

In [244]:
nys_denormalized.columns

Index(['ENTITY_CD', 'ENTITY_NAME', 'YEAR', 'NUM_TESTED', 'LEVEL1_COUNT',
       'LEVEL1_%TESTED', 'LEVEL2_COUNT', 'LEVEL2_%TESTED', 'LEVEL3_COUNT',
       'LEVEL3_%TESTED',
       ...
       'LEVEL1_COUNT_total_cohort_math_subgroup',
       'LEVEL1_%COHORT_total_cohort_math_subgroup',
       'LEVEL2_COUNT_total_cohort_math_subgroup',
       'LEVEL2_%COHORT_total_cohort_math_subgroup',
       'LEVEL3_COUNT_total_cohort_math_subgroup',
       'LEVEL3_%COHORT_total_cohort_math_subgroup',
       'LEVEL4_COUNT_total_cohort_math_subgroup',
       'LEVEL4_%COHORT_total_cohort_math_subgroup',
       'NOT_TESTED_total_cohort_math_subgroup',
       'NOT_TESTED_%COHORT_total_cohort_math_subgroup'],
      dtype='object', length=388)

In [256]:
nys_denormalized['ENTITY_NAME'].value_counts().head(5)

JOHN F KENNEDY MIDDLE SCHOOL    4
PARK AVENUE SCHOOL              4
JOHN F KENNEDY SCHOOL           4
LINCOLN SCHOOL                  3
GEORGE WASHINGTON SCHOOL        3
Name: ENTITY_NAME, dtype: int64

## Done with NYS, Now switching to NJ
- note: skipped all regents & accountability #s for NY, may come back to them
- also didn't us NY grad rate data, assume it's in the report card data as well.
- also didn't use NYC stats

In [188]:
# nj tables
nj_tables = [x for x in os.listdir('../prepared_data/') if x.find('nj_') > -1]

In [190]:
# re-define function to generate a shortname from tablename:
def shortname(s, kind ='nys'):
    if kind == 'nys':
        _ = s.lower()
        strings_to_replace= ['(',')','&','-','nys_reportcard_','.csv','results']
        for st in strings_to_replace:
            _ = _.replace(st, '')
        _ = '_'.join(_.split('_')[:4])
        if _[-1] == '_':
            _ = _[:-1]
        return _
    elif kind == 'nj':
        strings_to_replace= ['nj_','.csv']
        for st in strings_to_replace:
            s = s.replace(st, '')
        return s

In [ ]:
if 'nj_denormalized' in vars():
    del nj_denormalized

tables_to_skip = [
    'SchoolHeader',
    'EnrollmentTrendsByStudentGroup',
    'EnrollmentByRacialEthnicGroup',
    
]

# init with school names
nj_denormalized = pd.read_csv('../prepared_data/nj_SchoolHeader.csv')

for t in nj_tables:
    table_shortname = shortname(t)
    _df = pd.read_csv('../prepared_data/' + t)
    
    # logic to unstack common table patterns:
    if table_shortname == 'EnrollmentTrendsByGrade'or table_shortname == 'PreKAndK-FullDayHalfDay':
        _df = _df.pivot_table(
            values = 'Count', 
            index = ['CountyCode','DistrictCode','SchoolCode'], 
            columns=['Grade']
        ).reset_index()
    
    # data cleaning for all tables
    _df.drop_duplicates(inplace=True)
    
    # table skipping
    if list_contains(tables_to_skip, table_shortname):
        print('skipping table ' + table_shortname)
    else:
        print('trying to join ' + table_shortname )
        _tmp = nj_denormalized.merge(
            right = _df, 
            how = 'outer',
            on = ['CountyCode','DistrictCode','SchoolCode'],
            suffixes = ['', '_' + table_shortname]
        )
        nj_denormalized = _tmp
    del _df
print('all tables joined!')

In [269]:
_df = pd.read_csv('../prepared_data/nj_PreKAndK-FullDayHalfDay.csv')

In [270]:
_df.head()

,CountyCode,DistrictCode,SchoolCode,Grade,Count
0,1,10,60,KG - Full Day,90
1,1,10,60,KG - Half Day,0
2,1,10,60,PK - Full Day,25
3,1,10,60,PK - Half Day,0
4,1,110,30,KG - Full Day,62
